# <center> **EDA & CLEANING**

## **I) IMPORTS & EXPLORATION**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

### **1) DATASETS URLS**

In [2]:
namebasics_url = "https://datasets.imdbws.com/name.basics.tsv.gz"
akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
crew_url = "https://datasets.imdbws.com/title.crew.tsv.gz"
principals_url = "https://datasets.imdbws.com/title.principals.tsv.gz"
ratings_url ="https://datasets.imdbws.com/title.ratings.tsv.gz"

### **2) LOADING DATASETS & CLEANING**

#### **2.1 ratings dataset**

In [5]:
# ratings dataset loading: 
ratings = pd.read_csv(ratings_url, sep='\t', na_values='\\N', dtype={'averageRating': float, 'numVotes': int, 'tconst': str})
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2187
1,tt0000002,5.5,307
2,tt0000003,6.5,2276
3,tt0000004,5.1,196
4,tt0000005,6.2,3013


In [ ]:
# Exploration of ratings dataset
print(ratings.info())
print(ratings.duplicated().sum())
print(ratings.isnull().sum())
ratings['numVotes'].describe().round(2)
ratings['averageRating'].describe().round(2)

#### **2.2 basics dataset**

In [ ]:
# basics dataset loading using chunking to optimize memory usage
# use a separate name for the chunksize to avoid confusion with each chunk dataframe
chunksize = 1_000_000
filtered_basics = []

# define current_year 
current_year = globals().get('current_year', pd.Timestamp.now().year)

usecols = ['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres']

for chunk_df in pd.read_csv(basics_url,
                            sep='\t',
                            na_values='\\N',
                            usecols=usecols,
                            dtype={'tconst': str,
                                   'titleType': str,
                                   'primaryTitle': str,
                                   'originalTitle': str,
                                   'isAdult': 'Int64',
                                   'startYear': 'Int64',
                                   'endYear': str,
                                   'runtimeMinutes': str,
                                   'genres': str
                                  }, 
                            chunksize=chunksize):
    # ensure each comparison is parenthesized to avoid operator precedence issues
    mask = (
        (chunk_df['titleType'] == 'movie') &
        (chunk_df['isAdult'] == 0) &
        (chunk_df['startYear'] >= 1980) &
        (chunk_df['startYear'] <= current_year)
    )
    filtered_chunk = chunk_df[mask]
    filtered_basics.append(filtered_chunk)

basics = pd.concat(filtered_basics, ignore_index=True)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,0,2019,NaN,NaN,"Action,Crime"
1,tt0015724,movie,Dama de noche,Dama de noche,0,1993,NaN,102,"Drama,Mystery,Romance"
2,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
3,tt0036606,movie,"Another Time, Another Place","Another Time, Another Place",0,1983,NaN,118,"Drama,War"
4,tt0038086,movie,Shiva und die Galgenblume,Shiva und die Galgenblume,0,1993,NaN,NaN,Thriller


#### **2.3 akas dataset**

In [ ]:
# akas dataset loading using chunking to optimize memory usage
chunksize = 1_000_000
filtered_akas = []

for chunk_df in pd.read_csv(akas_url,
                            sep='\t',
                            na_values='\\N',
                            dtype={'titleId': str,
                                   'ordering': int,
                                   'title': str,
                                   'region': str,
                                   'language': str,
                                   'types': str,
                                   'attributes': str,
                                   'isOriginalTitle': 'Int64'},
                            chunksize=chunksize):
    # filter for FR region and original titles
    mask = (
        (chunk_df['region'] == 'FR') &
        (chunk_df['titleId'].isin(basics['tconst'])) )
    
    filtered_chunk = chunk_df[mask]
    filtered_akas.append(filtered_chunk)

akas = pd.concat(filtered_akas, ignore_index=True)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0035423,30,Kate et Léopold,FR,NaN,imdbDisplay,NaN,0
1,tt0036606,11,Les Coeurs captifs,FR,NaN,imdbDisplay,NaN,0
2,tt0038687,7,Que la lumière soit,FR,NaN,imdbDisplay,NaN,0
3,tt0048550,2,Le rendez-vous des quais,FR,NaN,imdbDisplay,NaN,0
4,tt0059325,5,Génération 45,FR,NaN,imdbDisplay,NaN,0


In [ ]:

# We keep and export the cleaned datasets: ratings and principals (distinguish actors/actresses and directors)
# Merge basics and akas into a new dataframe called movies 
movies = pd.merge(basics, akas, left_on='tconst', right_on='titleId', how='inner')
# we keep only useful columns
movies = movies[['tconst', 'primaryTitle', 'originalTitle', 'startYear', 'runtimeMinutes', 'genres', 'region', 'title']]
movies.head()
movies.info()

In [ ]:
# define a function to detect language safely in movies
from langdetect import detect, LangDetectException
def detect_language_safe(text):
    """
    Attempts to detect language. Returns 'unknown' if it fails.
    """
    try:
        # Check if text is a string and has at least one letter
        if isinstance(text, str) and any(c.isalpha() for c in text):
            return detect(text)
        else:
            return 'unknown'
    except LangDetectException:
        return 'unknown'

In [28]:
movies['detected_language'] = movies['originalTitle'].apply(detect_language_safe)

In [ ]:
# filter movies to keep only those detected as French or English 
# filter Holywood blucksbusters (vote_count + average_rating criteria can be added later)
movies = movies[movies['detected_language'].isin(['fr', 'en'])]
movies.info()

#### **2.4 principals dataset**

In [35]:
# principals dataset loading using chunking to optimize memory usage
chunksize = 1_000_000
filtered_principals = []
for chunk_df in pd.read_csv(principals_url, 
                            sep='\t',
                            na_values='\\N',
                            dtype={'tconst': str,
                                   'ordering': int,
                                   'nconst': str,
                                   'category': str,
                                   'job': str,
                                   'characters': str},
                            chunksize=chunksize):
    # filter for titles present in the movies dataframe
    mask = chunk_df['tconst'].isin(movies['tconst']) & chunk_df['category'].isin(['actor', 'actress', 'director'])
    filtered_chunk = chunk_df[mask]
    filtered_principals.append(filtered_chunk)
principals = pd.concat(filtered_principals, ignore_index=True)
principals.head()

,tconst,ordering,nconst,category,job,characters
0,tt0035423,1,nm0000212,actress,NaN,"[""Kate McKay""]"
1,tt0035423,2,nm0413168,actor,NaN,"[""Leopold""]"
2,tt0035423,3,nm0000630,actor,NaN,"[""Stuart Besser""]"
3,tt0035423,4,nm0005227,actor,NaN,"[""Charlie McKay""]"
4,tt0035423,5,nm0005169,actress,NaN,"[""Darci""]"


#### **2.4 namebasics dataset**

In [36]:
# namebasics dataset loading using chunking to optimize memory usage
chunksize = 1_000_000
filtered_namebasics = []
for chunk_df in pd.read_csv(namebasics_url, 
                            sep='\t',      
                            na_values='\\N',
                            dtype={'nconst': str,
                                   'primaryName': str,
                                   'birthYear': 'Int64',    
                                   'deathYear': 'Int64',
                                   'primaryProfession': str,
                                   'knownForTitles': str},
                            chunksize=chunksize):
    # filter for nconst present in principals dataframe
    mask = chunk_df['nconst'].isin(principals['nconst'])
    filtered_chunk = chunk_df[mask]
    filtered_namebasics.append(filtered_chunk)
namebasics = pd.concat(filtered_namebasics, ignore_index=True)
namebasics.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0027125,tt0025164"
1,nm0000002,Lauren Bacall,1924,2014,"actress,miscellaneous,soundtrack","tt0037382,tt0075213,tt0038355,tt0117057"
2,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
3,nm0000008,Marlon Brando,1924,2004,"actor,director,writer","tt0078788,tt0068646,tt0047296,tt0044081"
4,nm0000009,Richard Burton,1925,1984,"actor,producer,director","tt0061184,tt0087803,tt0057877,tt0059749"


In [37]:
# we merge principals with namebasics 
principals = pd.merge(principals, namebasics, on='nconst', how='left')
principals = principals[['tconst', 'nconst', 'category', 'primaryName', 'birthYear', 'deathYear']]
principals.head()

,tconst,nconst,category,primaryName,birthYear,deathYear
0,tt0035423,nm0000212,actress,Meg Ryan,1961,<NA>
1,tt0035423,nm0413168,actor,Hugh Jackman,1968,<NA>
2,tt0035423,nm0000630,actor,Liev Schreiber,1967,<NA>
3,tt0035423,nm0005227,actor,Breckin Meyer,1974,<NA>
4,tt0035423,nm0005169,actress,Natasha Lyonne,1979,<NA>


## **II) FINAL DATASETS** 

In [38]:
directors = principals[principals['category'] == 'director']
actors = principals[principals['category'].isin(['actor', 'actress'])]
genres = movies[['tconst', 'genres']].copy()
genres['genres'] = genres['genres'].str.split(',')
genres = genres.explode('genres').reset_index(drop=True)

In [39]:
# dataframes to export for PowerBI analysis 
genres.info()
directors.info()
actors.info()
ratings.info()
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61333 entries, 0 to 61332
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tconst  61333 non-null  object
 1   genres  60403 non-null  object
dtypes: object(2)
memory usage: 958.5+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 33036 entries, 10 to 241978
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tconst       33036 non-null  object
 1   nconst       33036 non-null  object
 2   category     33036 non-null  object
 3   primaryName  33036 non-null  object
 4   birthYear    16450 non-null  Int64 
 5   deathYear    2925 non-null   Int64 
dtypes: Int64(2), object(4)
memory usage: 1.8+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 208943 entries, 0 to 241977
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   tconst       208943 non-null 

## **III) EXPORT FOR VISUALIZATION** 

In [40]:
export_path = "C://Users/barba/Case_studies/Cinema_recommender/cleaned_data"
for df, name in zip([movies.drop('genres', axis=1), ratings, directors, actors, genres], 
                    ['movies', 'ratings', 'directors', 'actors', 'genres']):
    file_path = os.path.join(export_path, f"{name}.csv")
    df.to_csv(file_path, index=False)  